# Tutorial 2: Bayesian Optimization

In this tutorial notebook we'll implement all the basic components of Bayesian optimization (BO), and see how to use BO for some sample functions.

## 0. Glossary and General Introduction

### 0.1 Bayes' Theorem

First, let's start with the absolute basics of Bayesian inference, the Bayes' theorem
    $$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

- $P(A|B)$ is the _posterior probablity_ of event $A$ given that event B is observed.
- $P(B|A)$ is the _likelihood function_ of A given B; it is also the conditional probablity of observing $B$ given $A$.
- $P(A), P(B)$ are the _prior probabilities_ of observing $A$ and $B$, also known as _marginal probability_

For most applications, the observed event $B$ is already fixed and we can consider $P(B)$ simply as a constant. Thus the Bayes' theorem reads
    $$P(A|B) \propto P(B|A) P(A) $$,
i.e. the posterior probability is proportional to priors times likelihood.

### 0.2 Gaussian Process

A Gaussian process (GP) is a stochastic process (a joint distribution of infinitely many random variables), which can be used as a probablistic model (surrogate model) for the objective function in regression and classification tasks.

A GP can be fully described by it's mean $\mu$ and covariance function $k(\cdot,\cdot)$
    $$f(x) \sim \mathcal{GP}(\mu(x), k(x,x'))$$

- __Gaussian process regression (GPR)__, also formerly known as _kriging_: a method to interpolate a unkown objective function.
- __Prior mean__ $\mu(x)$: basic building block of GP; prior belief on the averaged objective function values, usually set to a constant if the function behaivor is unknown.
- __Kernel__ $k(\cdot,\cdot)$, also known as the _covariance function_ $cov(\cdot,\cdot)$: basic building block of GP; prior belief on the characteristics of the unkown function.

#### 0.2.1 Common Kernels used in GP

Some of the commonly used kernels are listed below. They can also be combined to build more complex kernels representing the underlying physics of the objective function.

- Linear: $k_\mathrm{L}(x,x') = x^\intercal x'$
- __White Gaussian noise__: $k_\mathrm{n}(x,x')=\sigma_{n}^2 \delta_{x,x'}$ , i.e. a diagnal noise term with Gaussian noise $\sigma_{n}^2$.
- __Radial basis function (RBF)__, also known as squared exponential (SE): $k_{\mathrm{RBF}} (x,x') = \exp(-\frac{||x-x'||^2}{2 l^2})$ , where $l$ is the length scale, see below. This resembles a normal Gaussian distribution.
- Matérn: $k_\mathrm{Matern} (x,x') = \frac{2^{1-\nu}}{\Gamma(\nu)} \left( \sqrt{2\nu ||x-x'||} / l \right)^\nu K_\nu \left( \sqrt{2\nu ||x-x'||} / l \right) $, where $\Gamma$ is the gamma function, $K_\nu$ the modified Bessel function and $\nu$ the parameter of the Matérn kernel. Common choices are $\nu=\frac{3}{2},\frac{5}{2},...$
- Ornstein-Uhlenbeck: $k_\mathrm{OU}(x,x')=\exp(-||x-x'||/l)$


#### 0.2.2 GP Hyperparamters 

The characteristics of GP, or its ability to approximate the unknown function are dependent both on __the choice of the covariance function__ and the __values of the hyperparamters__. The hyperparamters are usually either choosen manually based on the physics, or obtained from the maximum likelihood fit (log-likelihood fit, maximum a posteriori fit) during the optimization.

- __Lengthscale__ $l$:


### 0.3 Bayesian Optimization

Below are some general terms used in the BO field, some of which are often used interchangeably.

- __Acquisition function__ $\alpha$: is built on the GP posterior, controls the behavior of optimization. For the standard verison of BO, the next sample point is chosen at $\mathrm{argmax}(x)$. In this tutorial we will introduce two widely used acquisition functions: the expected improvment (EI) and the upper confidence bound (UCB).
- __Objective__, metric, or target function: a unknown (black-box) function, for which the value is to be optimized (here: maximization).
- __Search space__, bounds, or optimization range: A (continous) parameter space where the input parameters are allowed to be varied in the optimization.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, DotProduct, WhiteKernel
from sklearn.datasets import make_friedman2
from skopt import gp_minimize
from scipy.optimize import minimize
from utils.gp_helper import plot_gp, plot_gp_with_acq, plot_bo_result, Acquisition, AcqEI, AcqUCB

In [ ]:
rng = np.random.default_rng(1)

## 1. Build a Gaussian Process

Here we will use the [scikit-learn](https://scikit-learn.org/stable/modules/gaussian_process.html) to build a Gaussian process model.
In the end of this notebook you will also find a incomplete set of other commonly used GP and BO projects.

In [ ]:
# Define a RBF kernel (covariance function)
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
# Initialize a GP model
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

## 2. Define an Unkown Target Function

In [ ]:
def f_target(x, noise=0.1, rng = np.random.default_rng()):
    f = (-1.4 + 3.0 * x) * np.sin(18.0 * x)
    return f + rng.random(x.shape) * noise

In [ ]:
# Let's visualize it
xlow, xhigh = 0, 1
x = np.linspace(xlow,xhigh,200)
y = f_target(x, noise=0)
x_samples = rng.uniform(low=xlow,high=xhigh,size=6)
y_samples = f_target(x_samples, noise=0.1, rng=rng)

plt.plot(x,y,label='True f')
plt.plot(x_samples,y_samples,"*",label="Noisy Samples")
plt.legend()

Feel free to change the length_scale below to see the difference in resulted GP posteriors.

One can use `kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))` to also fit the length_scale within the specified range.

In [ ]:
# define GP model
kernel = 1 * RBF(length_scale=0.1, length_scale_bounds="fixed") + WhiteKernel(noise_level=0.1)
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

In [ ]:
# Visualize the GP fitted to the target function
fig, axes = plt.subplots(1,2, figsize=(8,3))
ax1, ax2 = axes

# plot GP prior
ax1.set_title("Prior")
plot_gp(gpr, x, y, x_samples, y_samples, ax=ax1)

# fit GP to the function and plot posterior2
gpr.fit(x_samples.reshape(-1,1),y_samples)
ax2.set_title("Posterior")
plot_gp(gpr, x, y, x_samples, y_samples, ax=ax2)

fig.tight_layout()
ax1.legend()

## 3. Maximization of the acquisition function

Here we will implement two acquisition functions: 

the __expected improvement (EI)__
$$ \begin{split}   
\alpha_{\text{EI}} (x)  &= \mathbb{E} [ \max (\mu (x)-(f_{\text{best}}+\xi),0)  ] \\  
&= (\mu(x)-(f_{\text{best}}+\xi)) \Phi (Z) + \sigma (x)\phi (Z) 
\end{split}, 
$$
where
$$
\begin{split}
         Z  & = \frac{\mu(x)-(f_{\text{best}}+\xi)}{\sigma(x)},
\end{split}
$$

and the __upper confidence bound (UCB)__
$$
    \alpha_{\text{UCB}} (x) = \mu(x) + \kappa \sigma (x).
$$

See `utils/gp_helper.py` for the implementation of the acquisition fucntions.

__Exploration-exploitation tradeoff__: the behaviour of the acquisition fucntions are controlled via the hyperparameters $\xi$ and $\kappa$. Larger values lead to more exploration and vice versa.

In [ ]:
# Define a EI acquisition
acq_EI = AcqEI(xi=0.)  # feel free to change the value of xi to see the different behaviour of EI

# Calculate the acquisition function
y_acq = acq_EI.get_acq(x.reshape(-1,1), gpr)


# Visualize the acquisition functions 

fig, axes = plt.subplots(2,1,figsize=(5,4),gridspec_kw={"height_ratios": [2,1]})
ax1, ax2 = axes
ax2.set_ylabel('EI')

plot_gp_with_acq(gpr, x, y, x_samples, y_samples, y_acq, axes=axes, fig=fig)


In [ ]:
# Define a UCB acquisition
acq_UCB = AcqUCB(k=2)  # feel free to change the value of xi to see the different behaviour of EI

# Calculate the acquisition function
y_acq = acq_UCB.get_acq(x.reshape(-1,1), gpr)

# Visualize the acquisition functions 

fig, axes = plt.subplots(2,1,figsize=(5,4),gridspec_kw={"height_ratios": [2,1]})
ax1, ax2 = axes
ax2.set_ylabel('UCB')

plot_gp_with_acq(gpr, x, y, x_samples, y_samples, y_acq, axes=axes, fig=fig)

## 4. Maximize the Target Function

_Note_: Here the code is used to maximize a unknown target fucntion. For minimization tasks, one can simply multiply -1 to the target function.

Some helper functions are imported to plot the progress of BO, see `utils/gp_helper.py`

The most simple form of Bayesian optimization can be divided in following steps:

1. Initialize GP model $\mathcal{GP}(\mu, \sigma)$
2. Build acquisition function $\alpha$
3. Sample next point $x_i$ of target function $f$ at $\text{argmax}(\alpha)$
4. Observe $y_i = f(x_{i})$ and refit GP model.


In [16]:
# Define a simple Bayesian optimizer  

def bayesian_optimize(gpmodel: GaussianProcessRegressor, acquisition: Acquisition, target_func, init_points=None, steps=50, bounds=None, xdim=1, rng=np.random.default_rng()):
    # for logging purpose
    history = {
        "X_init": [],
        "Y_init": [],
        "X": [],
        "Y": [],
        "X_best": [],
        "Y_best": [],
        }

    # Initial samples to start with 
    if init_points is None:
        init_points = rng.uniform(low=bounds[:, 0], high=bounds[:, 1], size=(3, xdim))
    y_init = []
    for xi in init_points:
        y_init.append(target_func(xi))
    y_init = np.array(y_init)
    history["X_init"] = init_points
    history["Y_init"] = y_init
    n_init = init_points.shape[0]

    X = init_points.reshape(-1,xdim)
    Y = y_init.reshape(-1,1)
    
    # Actual optimization step
    for _ in range(steps):
        # fit gp model
        gpmodel.fit(X, Y)
        # maximize acquisition
        x_next = acquisition.suggest_next_sample(gpmodel, bounds=bounds)
        # sample at argmax(acquisition)
        y_next = target_func(x_next)
        # augment data
        X = np.vstack([X, x_next.reshape(-1,xdim)])
        Y = np.vstack([Y, y_next.reshape(-1,1)])
    
    # log process
    history["X"] = X[n_init:]
    history["Y"] = Y[n_init:]
    i_best = Y.argmax()
    history["X_best"] = X[i_best]
    history["Y_best"] = Y[i_best]
    
    return gpmodel, history


In [ ]:
# Start Optimization [Feel free to play around with the parameters in this block and observe effects]

# define a GP model
kernel = 1 * RBF(length_scale=0.5, length_scale_bounds=[1e-2,1]) + WhiteKernel(noise_level=0.1) # allow automatic determination of length-scale
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
# define a Acquisition function
myacq = AcqEI(xi=0.001)
# start BO
bounds = np.array([[0,1]])
nsteps=30
gpr, history = bayesian_optimize(gpr, myacq, target_func=f_target, steps=nsteps, bounds=bounds, rng=rng)


In [ ]:
# Visualize Results
fig, ax = plt.subplots()
x_fine = np.linspace(0,1,500)
y_fine = f_target(x_fine,noise=0)
ymax = np.ones(nsteps)*y_fine.max()
ax.plot(ymax, ls='--',color='gray', label='True maximum')
ax.fill_between(np.arange(nsteps), ymax-0.1, ymax+0.1, alpha=0.3, color='gray')
ax.plot(history["Y"], label="BO result")
ax.legend()
ax.set_xlabel("Steps")
ax.set_ylabel("y")

## Performance benchmark (Optional Module)

against "classical" algorithms:
- Nelder-Mead
- Newtonian methods

### Define a more complex function

Let's use the Ackley function:
$$
    f(x_1,x_2) = -20 \exp \left[-0.2 \sqrt{0.5(x_1^2+x_2^2)} \right] - \exp \left[0.5(\cos(2\pi x_1) + \cos(2\pi x_2)) \right] + e + 20
$$

![Ackley function](img/ackley.png)


In [19]:
# Define more complex functions
def f_ackley(x: np.ndarray, yhist = None) -> np.ndarray:
    """func_log: used to track the optimization progress"""
    assert x.shape[1] ==2
    y = -20 * np.exp(-0.2 * np.sqrt(0.5 * np.sum(np.square(x), axis=1))) 
    y -= np.exp(0.5*(np.cos(2*np.pi*x[:,0]) + np.cos(2*np.pi*x[:,1])))
    y += np.e + 20
    y *= -1
    if yhist is not None:
        yhist.append(y)
    return y

### Start optimization with different algorithms

In [13]:
bounds_ackley = np.array([[-5,5],[-5,5]])
n_restart = 10  # try 10 times for each algorithm
maxiter = 50

In [20]:
# Nelder Mead
nm_hist = [0] * n_restart
for i in range(n_restart):
    rng = np.random.default_rng(i)
    x0 = rng.uniform(low=bounds_ackley[:,0], high=bounds_ackley[:,1])
    nm_hist[i] = []
    res = minimize(
        lambda x: -1 * float(f_ackley(np.array(x).reshape(-1,2), nm_hist[i])),
        x0 = x0,
        bounds=bounds_ackley,
        options={'maxfev': maxiter, 'xatol': 1e-10},
        method='Nelder-Mead'
        )

In [22]:
%%capture
# BO our version: more exploitation
bo_hist_exploit = [0] * n_restart
kernel = 1 * RBF(length_scale=0.3, length_scale_bounds=[1e-2,1]) #
for i in range(n_restart):
    rng = np.random.default_rng(i)
    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    myacq = AcqUCB(k=0.1)
    bo_hist_exploit[i] = []
    _, _ = bayesian_optimize(
        gpr, myacq, target_func=lambda x: -1 * f_ackley(np.array(x).reshape(-1,2), bo_hist_exploit[i]), steps=maxiter, bounds=bounds_ackley, rng=rng, xdim=2
        )

In [ ]:
# BO our version: more exploration
bo_hist_explore = [0] * n_restart
kernel = 1 * RBF(length_scale=0.3, length_scale_bounds=[1e-2,1]) #
for i in range(n_restart):
    rng = np.random.default_rng(i)
    gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    myacq = AcqUCB(k=2.0)
    bo_hist_explore[i] = []
    _, _ = bayesian_optimize(
        gpr, myacq, target_func=lambda x: -1 * f_ackley(np.array(x).reshape(-1,2), bo_hist_explore[i]), steps=maxiter, bounds=bounds_ackley, rng=rng, xdim=2
        )

In [ ]:
%%capture  
# suppress output

# Default sklearn version of BO
skopt_hist = [0] * n_restart
for i in range(n_restart):
    skopt_hist[i] = []
    res = gp_minimize(lambda x: -1 * float(f_ackley(np.array(x).reshape(-1,2), skopt_hist[i])),dimensions=bounds_ackley,n_calls=maxiter, random_state=i)

### Compare their performance

In [ ]:
# plot the results
fig,ax = plt.subplots()
plot_bo_result(nm_hist, ax, label='Nelder-Mead')
plot_bo_result(skopt_hist, ax, label='scikit-BO')
plot_bo_result(bo_hist_explore, ax, label='BO explore')
plot_bo_result(bo_hist_exploit, ax, label='BO exploit')
ax.legend()

---



## What's next?

Hopefully you have learned something about BO, if you want to try it yourself afterwards, below are some interesting resources.


### Publication: Various applications of BO in accelerator physics

- Applications in LPA

### Books and papers on Bayesian optimization in general

- C. E. Rasmussen and C. K.I. Williams [Gaussian Processes for Machine Learning](https://gaussianprocess.org/gpml/): __the__ classic textbook of Gaussian process
- 


### Bayesian Optimization / Gaussian Process packages in python

Below is a incomplete selection of python packages for BO and GP, each with its own strength and drawback.

- [scikit-learn Gaussian processes](https://scikit-learn.org/stable/modules/gaussian_process.html#) : recommended for sklearn users, not as powerful as other packages.
- [GPyTorch](https://gpytorch.ai/) : a rather new package implemented natively in PyTorch, which makes it very performant. Also comes with a Bayesian optimization package [BOTorch](https://botorch.org/), offering a variety of different optimization methods (mult-objective, parallelization...). Both packages are being actively developed maintained; recommended for PyTorch users.
- [GPflow](https://www.gpflow.org/) : a GP package implemented in TensorFlow, it also has a large community and is being actively maintained; The new BO package [Trieste](https://secondmind-labs.github.io/trieste) is built on it.
- [GPy](http://sheffieldml.github.io/GPy/) from the Sheffield ML group : A common/classic choice for building GP model, includes a lot of advanced GP variants; However in recent years it is not so actively maintained. It comes with the accompanying Bayesian optimization package [GPyOpt](https://github.com/SheffieldML/GPyOpt), for which the maintainance stoped since 2020.
- [Dragonfly] : a open-source BO package; offers also command line tool, easy to use if you are a practitioner. However if one has less freedom to adapt and expand the code.

C.f. the [wikipedia page](https://en.wikipedia.org/wiki/Comparison_of_Gaussian_process_software#Comparison_table) for a more inclusive table with GP packages for other languages.